# Homology of Reachable Sets

Reachable set coordinator script: 
```bash
./scripts/federer_20240620.sh
```

Resulted in `../results/federer*/` folders. 

In [1]:
import glob

In [2]:
# get all files matching '../results/federer*/'
files = glob.glob('../results/federer_p*/')

print("Length of files: ", files)

# now get the {path}/Y_to_U.json and load 
# the json file and get the values of the keys
import json 
import os

file_to_Rt = {}

for file in files:
    print("File: ", file)
    json_file = os.path.join(file, 'R_t.json')
    print("json_file: ", json_file)
    with open(json_file) as f:
        data = json.load(f)
        file_to_Rt[file] = data

Length of files:  ['../results/federer_pool10_push0.1_randpoolfalse_2024-06-20-231034/', '../results/federer_pool10_push2.5_randpooltrue_2024-06-20-231034/', '../results/federer_pool10_push10.0_randpooltrue_2024-06-20-231034/', '../results/federer_pool10_push1.0_randpoolfalse_2024-06-20-231034/', '../results/federer_pool10_push10.0_randpoolfalse_2024-06-20-231034/', '../results/federer_pool10_push2.5_randpoolfalse_2024-06-20-231034/', '../results/federer_pool10_push0.1_2024-06-20-230901/', '../results/federer_pool10_push1.0_randpooltrue_2024-06-20-231034/', '../results/federer_pool100_push0.1_randpooltrue_2024-06-20-231034/', '../results/federer_pool10_push0.1_randpooltrue_2024-06-20-231034/']
File:  ../results/federer_pool10_push0.1_randpoolfalse_2024-06-20-231034/
json_file:  ../results/federer_pool10_push0.1_randpoolfalse_2024-06-20-231034/R_t.json
Data:  [1032, 1544, 1806, 16, 17, 1041, 19, 20, 18, 22, 23, 24, 1049, 3226, 1691, 9756, 21, 9378, 10148, 679, 10288, 11128, 2614, 2618, 

AttributeError: 'list' object has no attribute 'keys'